In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from time import time

from torch.utils.data import TensorDataset, DataLoader

from llama import Tokenizer
tokenizer = Tokenizer('./tokenizer.model')

#hyperparameters
n_embd = 256
block_size = 128
n_head = 4
n_layer = 4
vocab_size = 32_000
batch_size = 32
device = 'cuda' if torch.cuda.is_available() else 'cpu'
learning_rate = 1e-3

text = open('../data/shakespeare.txt', encoding="utf-8").read()

data = torch.tensor(tokenizer.encode(text, False, False))
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    X = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+1+block_size] for i in ix])
    X, y = X.to(device), y.to(device)
    return X, y

def get_all_data(split):
    
    data = train_data if split == "train" else val_data
    X = torch.stack([data[i:i+block_size] for i in range(len(data) - block_size)])
    y = torch.stack([data[i+1:i+1+block_size] for i in range(len(data) - block_size)])
    X, y = X.to(device), y.to(device)
    return X, y


X, y = get_all_data('train')
dataset = TensorDataset(X, y)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

class Head(nn.Module):
    
    def __init__(self, head_size):
        super().__init__()
        
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        
    def forward(self, x):
        
        B,T,C = x.shape
        
        k = self.key(x)
        q = self.query(x)
        
        wei = q @ k.transpose(-2, -1)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        
        v = self.value(x)
        out = wei @ v
        return out
    
class MultiHeadAttention(nn.Module):
    
    def __init__(self, head_size):
        super().__init__()
        
        self.heads = nn.ModuleList([Head(head_size) for _ in range(n_head)])
        self.proj = nn.Linear(n_embd, n_embd)
        
    def forward(self, x):
        
        x = torch.cat([h(x) for h in self.heads], dim=-1)
        x = self.proj(x)
        return x
    
class FeedForward(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(n_embd * 4, n_embd)
        )
        
    def forward(self, x):
        return self.net(x)
    
class Block(nn.Module):
    def __init__(self):
        super().__init__()
        
        head_size = n_embd // n_head
        
        self.sa = MultiHeadAttention(head_size)
        self.ffwd = FeedForward()
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
        
    def forward(self, x):
        
        x = x + self.ln1(self.sa(x))
        x = x + self.ln2(self.ffwd(x))
        
        return x
    
class Transformer(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block() for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
    def forward(self, idx, targets=None):
        
        B, T = idx.shape
        
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T).to(device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        
        if targets == None:
            loss = None
        else:
            
            B,T,C = logits.shape
            
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            
            # Create a mask where targets are not equal to -1 (pad_id)
            mask = targets != -1
            # Use this mask to filter out logits and targets that you don't want to consider for the loss
            logits_masked = logits[mask]
            targets_masked = targets[mask]

            # Calculate loss with the non-padded targets and logits
            loss = F.cross_entropy(logits_masked, targets_masked)
            
        return logits, loss
    
    def generate(self, idx, max_gen_len):
        
        initial_idx = idx
        
        out = torch.zeros(max_gen_len).to(device)
        
        for i in range(max_gen_len):
            
            idx_cond = idx[:, -block_size:]
            
            logits, loss = self(idx_cond)
            
            logits = logits[:, -1, :]
            
            probs = F.softmax(logits, dim=-1)
            
            pred = torch.multinomial(probs, 1)
            
            out[i] = pred.item()
            
            idx = torch.cat((idx, pred), dim=-1)
            
        out = torch.cat((initial_idx.squeeze(0), out), dim=-1)
            
        print(tokenizer.decode(out.long().tolist()))
            
    
        
        
    
model = Transformer().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)



In [17]:
def prepare_data(input_text, target_text, tokenizer, device):
    # Encode the input and target texts
    xb = torch.tensor(tokenizer.encode(input_text, False, False), dtype=torch.long, device=device)
    yb_full = torch.tensor(tokenizer.encode(target_text, False, False), dtype=torch.long, device=device)
    
    # Create input and target tensors of the same length with appropriate padding
    max_len = max(len(xb), len(yb_full))  # Find the maximum length to pad to
    xb_padded = torch.full((max_len,), tokenizer.pad_id, dtype=torch.long, device=device)
    yb_padded = torch.full((max_len,), tokenizer.pad_id, dtype=torch.long, device=device)
    
    # Place the input sequence at the start of the padded input tensor
    xb_padded[:len(xb)] = xb
    
    # Place the target sequence at the end of the padded target tensor
    yb_padded[-len(yb_full):] = yb_full
    
    return xb_padded, yb_padded

# Example usage
input_text = 'The best person is '
target_text = 'Clark Kent'

# Assume device and tokenizer are already defined
xb, yb = prepare_data(input_text, target_text, tokenizer, device)

# Training loop
epochs = 10
for epoch in range(epochs):
    optimizer.zero_grad()
    
    # Forward pass
    logits, loss = model(xb.unsqueeze(0), yb)
    
    # Print loss every epoch
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")
    
    # Backward pass and optimization
    loss.backward()
    optimizer.step()

Epoch 1/10, Loss: 8.546026229858398
Epoch 2/10, Loss: 3.5971813201904297
Epoch 3/10, Loss: 0.14582370221614838
Epoch 4/10, Loss: 0.07925567775964737
Epoch 5/10, Loss: 0.038786862045526505
Epoch 6/10, Loss: 0.024439102038741112
Epoch 7/10, Loss: 0.018809068948030472
Epoch 8/10, Loss: 0.012455814518034458
Epoch 9/10, Loss: 0.011048940941691399
Epoch 10/10, Loss: 0.007354409899562597


In [86]:
xb = torch.tensor(tokenizer.encode('The best person is ', False, False))
xb = xb.to(device)

yb = torch.tensor(tokenizer.encode('Clark', False, False))
pad_size = len(xb) - len(yb)
yb_padded = torch.cat([torch.tensor([tokenizer.pad_id] * pad_size), yb])
yb = yb_padded.long().to(device)

count = 0
epochs = 10

for epoch in range(epochs):
    
    logits, loss = model(xb.unsqueeze(0), yb)
    
#     if count % (epochs // 10) == 0:
    print(loss)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    count += 1
    if count == epochs:
        break

tensor(0.0003, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0003, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0003, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0003, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0003, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0003, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0003, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0003, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0003, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(0.0003, device='cuda:0', grad_fn=<NllLossBackward0>)


In [29]:
model.generate(idx=torch.tensor(tokenizer.encode('The best person is ', False, False)).unsqueeze(0).to(device), max_gen_len=5)

The best person is  Kentman:
Come


In [21]:
model.generate(idx=torch.tensor(tokenizer.encode('HERMIONE: The best person is ', False, False)).unsqueeze(0).to(device), max_gen_len=500)

HERMIONE: The best person is  Kentлист, hot;
And hate, or worse than with him be fly I will be the house of basinous he would in Corioli withal's do not a leis very begins to thine:
So though it in delay.

Than my fear.

CUTIO:
That provost!
O, rap a whipt,
Thou camest, tarry, justice, marry none should wear to thou, then; for her! What I have to conclude forth
and farther off.

pinty; go bar of doors, instruct not--
Wave, I am a very windows one that dead, sir.

So will make my brother;
Be ready like.

First, of itself to shape, shipp'll be husband, the sick to me
To-night? Comms enemies to my prosperous.
th do the tables's issue, ye'd.

That lift their gentle Norfolk, thou doom kins. Take true contract at his more, and eat me!
RCIUS:'s, thou sleep than if I drink,

Was as this is dispersed
Her
Is partly to the house:
A traitor was at, and these bitterly
He's that neither knave:
Say do treble Benvolts? law?
A cold corse: Troy with
Who please.
Or Ethiopleave as severe!
The people, whos

In [20]:
count = 0
epochs = 500

for xb, yb in dataloader:
    
    logits, loss = model(xb, yb)
    
    if count % (epochs // 10) == 0:
        print(loss)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    count += 1
    if count == epochs:
        break

tensor(6.5864, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.9951, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.6821, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.7778, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.7910, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.6421, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.7215, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.4154, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.6948, device='cuda:0', grad_fn=<NllLossBackward0>)
tensor(3.4787, device='cuda:0', grad_fn=<NllLossBackward0>)
